In [1]:
import pymongo
import json
import datetime
import pandas as pd
from pprint import pprint
from pymongo import MongoClient

client = MongoClient('mongo', 27017, username='admin', password='DataMan2019!')

db=client.UCLFinal

In [10]:
#COMAN/MBAPPE:
#impostiamo la ricerca per i nomi dei 2 bomber sopracitati, suddividendo la collezione relativa al match ogni 10 minuti di partita, compreso l'intervallo
agg=db.partita.aggregate([
   {'$match':{"$and":[{"timestamp":{"$gt":"2020-08-23 22:51:00","$lte":"2020-08-23 23:01:00"}}, #ultima ricerca della serie
    {"text":{'$regex': "[Mm]bapp[eè]"}}]}},  
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': '$minuto_partita',
             'minuto_effettivo':'$minuto_effettivo'
            
         }
     },
       { '$out' :'Mbappè_110120'}   ]    #ultima collezione della serie di collezioni
   
)



In [67]:
agg=db.partita.aggregate([
   {'$match':{"$and":[{"timestamp":{"$gt":"2020-08-23 22:51:00","$lte":"2020-08-23 23:01:00"}}, #ultima ricerca per coman
    {"text":{'$regex': "[Cc]oman"}}]}}, 
     {
       '$project':
         {
             'id':'$id',
             'user_location':'$user_location',
             'language':'$language',
             'timestamp':'$timestamp',
             'text':'$text',
             'hashtags':'$hashtags',
             'urls':'$urls',
             'user_mentions':'$user_mentions',
             'geo':'$geo',
             'coordinates':'$coordinates',
             'retweet_count':'$retweet_count',
             'retweeted':'$retweeted',
             'is_a_retweet':'$is_a_retweet',
             'user_followers':'$user_followers',
             'minuto_partita': '$minuto_partita',
             'minuto_effettivo':'$minuto_effettivo'
            }
     },
       { '$out' :'Coman_110120'}   ]   #ultima collezione per coman

)


In [88]:
data=["2020-08-23 21:11:00","2020-08-23 21:21:00","2020-08-23 21:31:00","2020-08-23 21:41:00","2020-08-23 21:51:00","2020-08-23 22:01:00","2020-08-23 22:11:00","2020-08-23 22:21:00","2020-08-23 22:31:00","2020-08-23 22:41:00","2020-08-23 22:51:00","2020-08-23 23:01:00"]

In [89]:
minuti=['0-10','10-20','20-30','30-40','40-intervallo','intervallo','intervallo-52','52-62','62-72','72-82','82-92','recupero-fine'] #si crea la lista dei minutaggi per poi inserirla nel dataset

In [90]:

pos=[]
neu=[]
neg=[]
punteggio=[]
lb=[]
ub=[]
stdev=[]
confint_score=[]

 #si creano queste liste vuote che saranno compilate mano a mano dal codice sottostante di sentiment analysis, ed unite per dare forma ad un dataset

In [102]:
#SENTIMENT ANALYSIS:

collezione = db.Mbappè_110120.find({"language":"en"})   #inseriamo tutti i subset e compiliamo i dataset, prima per mbappe e poi per coman
#si seleziona la lingua inglese per questa analisi
import re
import math
import numpy as np

from textblob import TextBlob

positive =0 
negative =0
neutral=0
totale=0

lista=[]

def clean_tweet(tweet): 
    ''' Utility function to clean tweet text by removing links, special characters using simple regex statements. ''' 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+://\S+)", " ", tweet).split())

def get_tweet_sentiment(tweet):

    global positive
    global negative
    global neutral
    global totale
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_tweet(tweet)) 
    # set sentiment 
    #print(analysis.sentiment.polarity)
    if analysis.sentiment.polarity > 0:
        lista.append(analysis.sentiment.polarity)
        positive=positive+1
        #print(analysis)
        totale=totale+analysis.sentiment.polarity
        return 'positive'

    elif analysis.sentiment.polarity == 0: 
        neutral=neutral+1
        #print(analysis)
        return 'neutral'
    else: 
        lista.append(analysis.sentiment.polarity)
        negative=negative+1
        totale=totale+analysis.sentiment.polarity
        #print(analysis)
        return 'negative'

for tweet in collezione:

    get_tweet_sentiment(tweet['text'])
    

#creo 4 liste da comporre in un dataset finale:

punt=totale/(positive+negative)
dev=math.sqrt(np.var(lista))
lower=punt-1.64*dev/(math.sqrt(positive+negative))  #intervallo della normale
upper=punt+1.64*dev/(math.sqrt(positive+negative))
confint=(round(lower,3),round(upper,3))

pos.append(positive)
neu.append(neutral)
neg.append(negative)
punteggio.append(punt)
lb.append(lower)
ub.append(upper)
stdev.append(dev)
confint_score.append(confint)

In [ ]:
Mbappe=pd.DataFrame({'data':data,'minuti':minuti,'positivi':pos,'negativi':neg,'neutrali':neu,'lower_bound':lb,'upper_bound':ub,'stdev':stdev,'punteggio':punteggio,'confint_score':confint_score})
Mbappe

In [ ]:
Coman=pd.DataFrame({'data':data,'minuti':minuti,'positivi':pos,'negativi':neg,'neutrali':neu,'lower_bound':lb,'upper_bound':ub,'stdev':stdev,'punteggio':punteggio,'confint-score':confint_score})
Coman

In [85]:
Coman["lower_bound"]=round(Coman["lower_bound"],3)
Coman["upper_bound"]=round(Coman["upper_bound"],3)
Coman["sentiment_score"]=round(Coman["punteggio"],3)
Coman["stdev"]=round(Coman["stdev"],3)
Coman=Coman.drop("punteggio",axis='columns')

In [104]:
Mbappe["lower_bound"]=round(Mbappe["lower_bound"],3)
Mbappe["upper_bound"]=round(Mbappe["upper_bound"],3)
Mbappe["sentiment_score"]=round(Mbappe["punteggio"],3)
Mbappe["stdev"]=round(Mbappe["stdev"],3)
Mbappe=Mbappe.drop("punteggio",axis='columns')

In [87]:
#esportare dati in tsv
Coman.to_csv("Coman_def",sep='\t',index=False)


In [106]:
Mbappe.to_csv("Mbappe_definitivo",sep='\t',index=False)